In [390]:
import pandas as pd 
import numpy as np
from scipy import stats as st

In [400]:
df = pd.read_csv('ds.csv', sep=';')

In [403]:
dict_sex = {'m': 'Male', 'f': 'Female'}
dict_platform = {6: 'Desktop', 7: 'Mobile'}
df = df.rename(columns={'platform_id': 'platform'})

In [404]:
df['gender'] = df['gender'].replace(dict_sex)
df['platform'] = df['platform'].replace(dict_platform)

In [405]:
df['time_stamp'] = pd.to_datetime(df['time_stamp'])

Приберемо з датасету всі дані, що передували початку спліт-тестування:

In [406]:
ds = df[df['time_stamp'] >= '2017-03-24 16:00:00'].copy()
ds = ds.sort_values(by='time_stamp')

In [407]:
ds['gender'].unique()

array(['Female', 'Male', ' '], dtype=object)

In [408]:
ds = ds.loc[ds['gender'] != ' ']

Додаємо колонку, де вказуємо, з якої групи користувач – з тестової (непарний sender_id) чи з базової (парний sender_id)

In [409]:
ds['group'] = ds['sender_id'].apply(lambda x: 'Test' if x % 2 else 'Base')

In [410]:
ds['segment'] = df['platform'] + ' ' + df['gender'] # Сегментуємо по ОС та гендеру

In [411]:
ds

,sender_id,platform,time_stamp,gender,reg_date,group,segment
295310,3217917298,Mobile,2017-03-24 16:00:00,Female,19.03.2017,Base,Mobile Female
558717,3205198691,Mobile,2017-03-24 16:00:00,Male,15.01.2017,Test,Mobile Male
558724,3205198691,Mobile,2017-03-24 16:00:00,Male,15.01.2017,Test,Mobile Male
558725,3205198691,Mobile,2017-03-24 16:00:00,Male,15.01.2017,Test,Mobile Male
198599,3205140908,Desktop,2017-03-24 16:00:00,Male,15.01.2017,Base,Desktop Male
...,...,...,...,...,...,...,...
277231,3219303136,Desktop,2017-03-26 23:59:00,Female,26.03.2017,Base,Desktop Female
277325,3219308377,Mobile,2017-03-26 23:59:00,Male,26.03.2017,Test,Mobile Male
277313,3219308377,Mobile,2017-03-26 23:59:00,Male,26.03.2017,Test,Mobile Male
665169,3212514032,Desktop,2017-03-26 23:59:00,Male,19.02.2017,Base,Desktop Male


In [412]:
# Створюємо колонку logs_amount — кількість лайків, зроблена користувачем
ds['logs_amount'] = ds.groupby(['sender_id'])['time_stamp'].transform('count')

#### Експортуємо датасет для подальшої візуалізації у Tableau

In [413]:
ds.to_csv('ds.csv')

#### Графік розподілу у Tableau показав, що розподіл кількості лайків до кількості юзерів, що їх здійснили, розподіляється за розподілом Пуассона, що має велику кількість ступенів свободи (degrees of freedom).  
(тут було багато слова "розподіл", так було потрібно...)

#### Для аналізу середніх двох вибірок застосуємо t-критерій Ст'юдента, використовуючи Two sample test.

In [414]:
ds1 = ds.copy()

In [415]:
ds1 = ds1.drop_duplicates('sender_id', keep='first') # Залишаємо в датасеті тільки унікальні sender_id

In [416]:
ds1

,sender_id,platform,time_stamp,gender,reg_date,group,segment,logs_amount
295310,3217917298,Mobile,2017-03-24 16:00:00,Female,19.03.2017,Base,Mobile Female,2
558717,3205198691,Mobile,2017-03-24 16:00:00,Male,15.01.2017,Test,Mobile Male,11
198599,3205140908,Desktop,2017-03-24 16:00:00,Male,15.01.2017,Base,Desktop Male,33
558760,3205486416,Desktop,2017-03-24 16:00:00,Male,17.01.2017,Base,Desktop Male,69
662671,3218607720,Mobile,2017-03-24 16:00:00,Male,23.03.2017,Base,Mobile Male,9
...,...,...,...,...,...,...,...,...
708635,3219310953,Mobile,2017-03-26 23:55:00,Female,26.03.2017,Test,Mobile Female,5
708631,3219310125,Mobile,2017-03-26 23:56:00,Male,26.03.2017,Test,Mobile Male,4
708629,3219309969,Mobile,2017-03-26 23:57:00,Female,26.03.2017,Test,Mobile Female,8
246247,3216583745,Mobile,2017-03-26 23:57:00,Female,12.03.2017,Test,Mobile Female,1


In [250]:
ds1.pivot_table(index='segment', columns='group', values='logs_amount', aggfunc='mean')

group,Base,Test
segment,,
Desktop Female,10.601036,11.345404
Desktop Male,19.582756,18.400631
Mobile Female,6.353191,6.650575
Mobile Male,15.984723,12.730395


In [ ]:
ds1.pivot_table(index='segment', columns='group', values='logs_amount', aggfunc='mean')

In [395]:
ds1.pivot_table(index='segment', columns='group', values='logs_amount', aggfunc=['sum', 'mean', 'std'])

sum              mean                   std           
group            Base   Test       Base       Test       Base       Test
segment                                                                 
Desktop Female   4092   4073  10.601036  11.345404  20.113093  22.031793
Desktop Male    25438  23332  19.582756  18.400631  37.506033  29.160017
Mobile Female    2986   2893   6.353191   6.650575  10.736361  10.352863
Mobile Male     25112  19643  15.984723  12.730395  58.452396  27.187606

#### Розрахуємо Standart Deviation, Average та розмір базової та тестової вибірок

In [426]:
ds_sample_size = ds1.groupby('group', as_index=False).agg({'logs_amount': 'count'}).rename(columns={'logs_amount': 'sample_size'})
test1
ds_sample_mean = ds1.groupby('group', as_index=False).agg({'logs_amount': 'mean'}).rename(columns={'logs_amount': 'mean'})
ds_sample_std = ds1.groupby('group', as_index=False).agg({'logs_amount': 'std'}).rename(columns={'logs_amount': 'std'})

In [427]:
ds_descriptive = ds_sample_size.merge(ds_sample_mean, on='group').merge(ds_sample_std, on='group')
ds_descriptive

In [428]:
ds_descriptive

,group,sample_size,mean,std
0,Base,3726,15.466452,44.782857
1,Test,3605,13.853259,26.287139


#### Висуваємо гіпотезу, що поведінка користувачів із запровадженням нової версії не змінилась
Н0: μ_base = μ_test

H1: μ_base != μ_test

#### Перетворюємо на numpy-масив вибірку з тестової та базової групи спліт-тестування:

In [431]:
sample_base = ds1[ds1['group'] == 'Base']['logs_amount'].to_numpy()
sample_test = ds1[ds1['group'] == 'Test']['logs_amount'].to_numpy()

#### Розраховуємо t-критерій та відповідний йому p-value:

In [432]:
st.ttest_ind(a=sample_1, b=sample_2, equal_var=False)

Ttest_indResult(statistic=1.888192331064846, pvalue=0.05904788688607345)

#### Отримали p-рівень значущості = 0.059, що не дозволяє нам відхилити нульову гіпотезу
На графіку, що ми отримали у Tableau, порівнюючи середні вибірок та співставляюxи довірчі інтервали, бачимо, що середнє генеральної сукупності базової групи з 95% ймовірністю знаходиться в проміжку \[14.028:16.904\], а середнє генеральної сукупності тестової версії у проміжку \[12.995:14.711\].

In [276]:
ds1.to_csv('ds1.csv')